In [1]:
## Import relevant libraries
import sys
import glob

sys.path.append('../py_files/')
import quadrop2 as qd


qd.set_plotting_style()

### Data pre-procesing

In [2]:
# # Example usage
# base_dir = "../../../Thomson Lab Dropbox/David Larios/activedrops/paper/paper-v2/fig3-assets/"
# qd.consolidate_images(base_dir)

In [3]:
# Example usage
data_path =  "../../../Thomson Lab Dropbox/David Larios/activedrops/paper/figures/fig4-assets/original_tiffs/"

calibration_curve_paths = sorted(glob.glob("../../../Thomson Lab Dropbox/David Larios/activedrops/calibration_curve/***ugml.tif"))


conditions_dict = {
    "BBB": "Pos0",
    "OBB": "Pos1",
    "BOB": "Pos2",
    "OOB": "Pos3",
    "BBO": "Pos4",
    "OBO": "Pos5",
    "BOO": "Pos6",
    "OOO": "Pos7",
}

# # Organize PosX folders into condition folders
qd.organize_conditions(data_path, conditions_dict)

# # Now run the existing functions to reorganize the tiffs and rename the folders
conditions, subconditions = qd.prepare_conditions(data_path)\

# conditions = ['K401_MTs', 'Kif3_MTs']

time_interval_list = [60, 150, 75, 16, 40, 75, 150, 9]   # time intervals in seconds between frames for each condition

print("Conditions:", conditions)
print("Subconditions:", subconditions)




Conditions: ['BBB', 'BBO', 'BOB', 'BOO', 'OBB', 'OBO', 'OOB', 'OOO']
Subconditions: ['Rep1']


In [ ]:
qd.reorgTiffsToOriginal(data_path, conditions, subconditions)


In [ ]:
time_interval_list = [60, 40, 75, 75, 150, 150, 16, 9]

### Generate movies

In [ ]:
conditions[6:]

In [4]:
# Call the function
qd.fluorescence_heatmap(
    data_path, 
    conditions[6:], 
    subconditions, 
    channel='cy5', 
    time_interval_list=[16, 9], 
    vmax=16, 
    skip_frames=2, 
    calibration_curve_paths=calibration_curve_paths, 
    show_scalebar=False,
    # custom_title='K401 (slow-sustained)'
    )

Processing OOO - Rep1: 100%|██████████| 675/675 [03:11<00:00,  3.52it/s]


In [ ]:
# Example usage
qd.create_movies(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    frame_rate=60,
    skip_frames=1
    )


In [ ]:
conditions[::]

In [ ]:
qd.create_combined_heatmap_movie_custom_grid(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    grid_rows=2, 
    grid_cols=4, 
    frame_rate=1,
    batch_size=50
    )


In [ ]:

# def process_image(args):
#     import matplotlib.patheffects as patheffects  # For scale bar text outline

#     # Unpack arguments, allowing for an optional custom_title argument
#     # If args has 14 elements, the last is custom_title; otherwise, custom_title is None/False
#     if len(args) == 14:
#         (image_file, output_directory_path, channel, slope, intercept, vmax, time_interval, i, show_scalebar, min_frame, skip_frames, condition, subcondition, custom_title) = args
#     else:
#         (image_file, output_directory_path, channel, slope, intercept, vmax, time_interval, i, show_scalebar, min_frame, skip_frames, condition, subcondition) = args
#         custom_title = False

#     # Read the image into a numpy array
#     intensity_matrix = io.imread(image_file)

#     if channel == "cy5":
#         # Normalize intensity matrix to range [0, 1] for cy5 channel
#         matrix_to_plot = intensity_matrix / 1000
#         label = 'Normalized Fluorescence Intensity'
#     else:
#         # Convert intensity values to protein concentration using the calibration curve
#         matrix_to_plot = calculate_protein_concentration_ug_ml(intensity_matrix, slope, intercept)
#         matrix_to_plot = matrix_to_plot / 27000 * 1E6
#         label = 'Protein concentration (nM)'

#     # Plot the heatmap with a larger figure size
#     fig, ax = plt.subplots(figsize=(16, 16))
#     im = ax.imshow(matrix_to_plot, cmap='gray', interpolation='nearest', vmin=0, vmax=vmax)

#     if show_scalebar:
#         plt.colorbar(im, ax=ax, label=label)
    
#     # Remove axes and make image fill whole size
#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
#     ax.spines['bottom'].set_visible(False)
#     ax.spines['left'].set_visible(False)
    
#     # Add title inside the image at top right, but ensure it doesn't go out of bounds
#     if custom_title:
#         title_text = str(custom_title)
#     else:
#         title_text = f"{condition} "

#     # Truncate the title if it's too long to fit in the image
#     # Estimate max characters based on font size and figure width
#     # This is a heuristic: adjust max_chars as needed for your use case
#     max_chars = 30  # Reduced for right alignment
#     if len(title_text) > max_chars:
#         title_text = title_text[:max_chars-3] + "..."

#     # --- Custom coloring for all 'O' (orange) and all 'B' (blue) in the title ---
#     def draw_colored_title_all(ax, text, x, y, fontsize=80):
#         import matplotlib.patheffects as patheffects
#         # Colors
#         default_color = 'white'
#         o_color = '#FFA500'  # orange
#         b_color = '#1E90FF'  # blue

#         # Split text into segments by character, assigning color
#         segments = []
#         for c in text:
#             if c.upper() == 'O':
#                 segments.append((c, o_color))
#             elif c.upper() == 'B':
#                 segments.append((c, b_color))
#             else:
#                 segments.append((c, default_color))

#         # Now, draw each segment with correct color, right-aligned
#         # We'll use a dummy text to get the total width, then draw each segment offset from the right
#         import matplotlib.transforms as mtransforms
#         renderer = fig.canvas.get_renderer()
#         # Compose the full text for width calculation
#         full_text = ''.join(seg[0] for seg in segments)
#         # Get the right-aligned anchor in axes coordinates
#         right_x_axes = x
#         y_axes = y
#         # Convert axes coords to display coords
#         right_disp, y_disp = ax.transAxes.transform((right_x_axes, y_axes))
#         # Now, measure the width of the full text in display coords
#         t = ax.text(0, 0, full_text, fontsize=fontsize, weight='bold', va='top', ha='left',
#                     path_effects=[patheffects.withStroke(linewidth=3, foreground='black', alpha=0.7)],
#                     color=default_color)
#         fig.canvas.draw()  # Needed to get correct bbox
#         bbox = t.get_window_extent(renderer=renderer)
#         total_width = bbox.width
#         t.remove()
#         # Now, draw each segment, right-aligned
#         offset = 0
#         for seg, color in reversed(segments):
#             # Measure width of this segment
#             t = ax.text(0, 0, seg, fontsize=fontsize, weight='bold', va='top', ha='left',
#                         path_effects=[patheffects.withStroke(linewidth=3, foreground='black', alpha=0.7)],
#                         color=color)
#             fig.canvas.draw()
#             bbox = t.get_window_extent(renderer=renderer)
#             seg_width = bbox.width
#             t.remove()
#             # Place this segment at (right_disp - offset - seg_width, y_disp)
#             # Convert back to axes coords
#             x_disp = right_disp - offset - seg_width
#             x_axes, _ = ax.transAxes.inverted().transform((x_disp, y_disp))
#             ax.text(x_axes, y_axes, seg, 
#                     transform=ax.transAxes, color=color, fontsize=fontsize, 
#                     weight='bold', va='top', ha='left',
#                     path_effects=[patheffects.withStroke(linewidth=3, foreground='black', alpha=0.7)],
#                     clip_on=True)
#             offset += seg_width

#     # Use the custom colored title function with larger font size
#     draw_colored_title_all(ax, title_text, 0.98, 0.98, fontsize=100)
    
#     # Add timer information at top left in white
#     time_hours = (i - min_frame) * time_interval * skip_frames / 3600
#     time_minutes = (i - min_frame) * time_interval * skip_frames / 60
    
#     ax.text(0.02, 0.99, f"{time_hours:.2f} h", 
#             transform=ax.transAxes, color='white', fontsize=38, 
#             weight='bold', va='top', ha='left',
#             path_effects=[patheffects.withStroke(linewidth=3, foreground='black', alpha=0.7)])
    
#     ax.text(0.02, 0.94, f"{time_minutes:.2f} min", 
#             transform=ax.transAxes, color='white', fontsize=38, 
#             weight='bold', va='top', ha='left',
#             path_effects=[patheffects.withStroke(linewidth=3, foreground='black', alpha=0.7)])

#     # Draw a 1mm scale bar (730 pixels) at the bottom right
#     scalebar_length_px = 730
#     scalebar_height = max(4, int(matrix_to_plot.shape[0] * 0.005))  # 4 pixels or 0.5% of image height
#     color = 'white' if np.mean(matrix_to_plot) < 0.5 * vmax else 'black'

#     # Coordinates for the scale bar
#     x_start = matrix_to_plot.shape[1] - scalebar_length_px - 40  # 40 px from right edge
#     x_end = matrix_to_plot.shape[1] - 40
#     y_pos = matrix_to_plot.shape[0] - 40  # 40 px from bottom

#     # Draw the scale bar as a thick line
#     ax.hlines(
#         y=y_pos, xmin=x_start, xmax=x_end, colors=color, linewidth=scalebar_height, zorder=10, alpha=0.9
#     )
#     # Add text label above the scale bar
#     ax.text(
#         (x_start + x_end) / 2, y_pos - 15, "1 mm", color=color, fontsize=18, ha='center', va='bottom', weight='bold', zorder=11,
#         path_effects=[patheffects.withStroke(linewidth=3, foreground='black' if color == 'white' else 'white', alpha=0.7)]
#     )

#     # Save the heatmap with no borders
#     heatmap_filename = f"heatmap_frame_{i}.png"
#     heatmap_path = os.path.join(output_directory_path, heatmap_filename)
#     plt.savefig(heatmap_path, bbox_inches='tight', pad_inches=0, dpi=200)
#     plt.close(fig)


In [ ]:
import cv2
import numpy as np
import os
import glob
from tqdm import tqdm

def extract_protein_name(file_path):
    """
    Extracts the protein name from the file path.
    Assumes the protein name is in the first two underscore-separated tokens,
    where the second token may have additional info (e.g. "-RT") which is removed.
    """
    # Extract the directory name from the path
    dir_name = os.path.basename(os.path.dirname(file_path))
    tokens = dir_name.split('_')
    if len(tokens) < 2:
        raise ValueError(f"Directory name {dir_name} does not have enough tokens to extract protein name.")
    token1 = tokens[0]
    token2 = tokens[1].split('-')[0]
    return f"{token1}_{token2}"

def synchronize_frames_from_directories(frames_dict, time_multiplier, grid_shape, channel, fps_output=30.0, output_file_path="combined_video.avi"):
    """
    Synchronizes multiple frame directories based on a common time axis.
    
    Parameters:
        frames_dict (dict): Keys are frame directory paths with wildcards (e.g., "path/to/frames/*.png"), 
                           values are frame intervals in seconds.
        time_multiplier (float): Multiplier to the base time step.
        grid_shape (tuple): (rows, columns) specifying the grid layout.
        fps_output (float): Output video playback frames per second.
        output_file_path (str): Directory for saving the output video.
    """
    # Convert frames_dict keys and values to lists
    frame_patterns = list(frames_dict.keys())
    frame_intervals = list(frames_dict.values())
    
    n_videos = len(frame_patterns)
    grid_cells = grid_shape[0] * grid_shape[1]
    if grid_cells < n_videos:
        raise ValueError("Grid shape is too small for the number of videos provided.")
    
    # Get actual frame files for each pattern and count them
    frame_files_per_video = []
    total_frames_list = []
    
    for pattern in frame_patterns:
        # Get all PNG files matching the pattern
        frame_files = glob.glob(pattern)
        if not frame_files:
            raise ValueError(f"No PNG files found matching pattern: {pattern}")
        
        # Sort frames by the actual frame number, not the filename string
        def extract_frame_number(filepath):
            filename = os.path.basename(filepath)
            # Extract number from "heatmap_frame_55.png" -> 55
            if filename.startswith("heatmap_frame_") and filename.endswith(".png"):
                number_str = filename[14:-4]  # Remove "heatmap_frame_" and ".png"
                try:
                    return int(number_str)
                except ValueError:
                    return 0
            return 0
        
        # Sort by frame number, not by filename string
        frame_files = sorted(frame_files, key=extract_frame_number)
        
        frame_files_per_video.append(frame_files)
        total_frames_list.append(len(frame_files))
    
    # Extract protein names for output naming.
    protein_names = [extract_protein_name(fp) for fp in frame_patterns]
    proteins_combined = "-".join(protein_names)
    
    # Build the final output file name, including time_multiplier and fps_output.
    out_dir = os.path.dirname(output_file_path)
    # Format time_multiplier and fps_output for filename (avoid decimal point if possible)
    tm_str = f"{int(time_multiplier)}" if int(time_multiplier) == time_multiplier else f"{time_multiplier}"
    fps_str = f"{int(fps_output)}" if int(fps_output) == fps_output else f"{fps_output}"
    final_output_file = os.path.join(
        out_dir, 
        f"{proteins_combined}_{channel}_synced_tm{tm_str}_fps{fps_str}.avi"
    )
    
    # Calculate total times and find maximum.
    total_times = [frames * interval for frames, interval in zip(total_frames_list, frame_intervals)]
    total_time_sync = max(total_times)
    
    # Compute time step.
    base_time_step = min(frame_intervals)
    time_step = base_time_step * time_multiplier
    
    # Determine number of output frames.
    num_output_frames = int(total_time_sync / time_step)
    
    # Calculate playback duration.
    playback_duration_sec = num_output_frames / fps_output
    print(f"Movie duration: {playback_duration_sec:.2f} seconds")
    
    # Get dimensions from first frame of first directory.
    first_frame = cv2.imread(frame_files_per_video[0][0])
    if first_frame is None:
        raise ValueError(f"Could not read first frame from {frame_files_per_video[0][0]}")
    
    h, w = first_frame.shape[:2]
    cell_height, cell_width = h, w
    
    # Calculate output dimensions.
    output_frame_height = grid_shape[0] * cell_height
    output_frame_width = grid_shape[1] * cell_width
    
    # Create video writer.
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(final_output_file, fourcc, fps_output, (output_frame_width, output_frame_height))
    
    # Process frames: for each time step, compute the corresponding frame from each directory.
    for i in tqdm(range(num_output_frames), desc="Processing frames", unit="frame"):
        current_time = i * time_step
        frames = []
        
        for idx, (frame_files, frame_interval) in enumerate(zip(frame_files_per_video, frame_intervals)):
            # Calculate which frame we need for this time.
            frame_idx = int(round(current_time / frame_interval))
            
            # Ensure frame index is within bounds.
            if frame_idx >= total_frames_list[idx]:
                frame_idx = total_frames_list[idx] - 1
            
            # Get the frame file path.
            frame_path = frame_files[frame_idx]
            
            # Read the frame.
            frame = cv2.imread(frame_path)
            if frame is None:
                # If frame doesn't exist, create a black frame.
                frame = np.zeros((cell_height, cell_width, 3), dtype=np.uint8)
            
            frame = cv2.resize(frame, (cell_width, cell_height))
            frames.append(frame)
        
        # Fill remaining grid cells with black frames.
        while len(frames) < grid_cells:
            black = np.zeros((cell_height, cell_width, 3), dtype=np.uint8)
            frames.append(black)
        
        # Combine frames into grid.
        grid_rows = []
        for r in range(grid_shape[0]):
            row_frames = frames[r * grid_shape[1] : (r + 1) * grid_shape[1]]
            row_combined = np.hstack(row_frames)
            grid_rows.append(row_combined)
        combined_frame = np.vstack(grid_rows)
        
        out.write(combined_frame)
    
    out.release()
    print(f"Output saved to: {final_output_file}")



# Example usage with your PNG frame directories:
frames_data = {
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/paper/figures/fig4-assets/original_tiffs/output_data/movies/BBB_Rep1_heatmaps_cy5/heatmap_frame_*.png": 240,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/paper/figures/fig4-assets/original_tiffs/output_data/movies/BBO_Rep1_heatmaps_cy5/heatmap_frame_*.png": 160,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/paper/figures/fig4-assets/original_tiffs/output_data/movies/BOB_Rep1_heatmaps_cy5/heatmap_frame_*.png": 150,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/paper/figures/fig4-assets/original_tiffs/output_data/movies/BOO_Rep1_heatmaps_cy5/heatmap_frame_*.png": 150,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/paper/figures/fig4-assets/original_tiffs/output_data/movies/OBB_Rep1_heatmaps_cy5/heatmap_frame_*.png": 300,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/paper/figures/fig4-assets/original_tiffs/output_data/movies/OBO_Rep1_heatmaps_cy5/heatmap_frame_*.png": 300,
    "../../../../Thomson Lab Dropbox/David Larios/activedrops/paper/figures/fig4-assets/original_tiffs/output_data/movies/OOB_Rep1_heatmaps_cy5/heatmap_frame_*.png": 16*2,
    "../../../../Thomson Lab Dropbox/David Larios/activedrops/paper/figures/fig4-assets/original_tiffs/output_data/movies/OOO_Rep1_heatmaps_cy5/heatmap_frame_*.png": 9*2,
}

# Now you can use much lower time_multiplier since no seeking issues:
time_multiplier = 1  # Maximum temporal resolution!
grid_shape = (1, 2)

synchronize_frames_from_directories(frames_data, time_multiplier, grid_shape, 
                                   channel="cy5", fps_output=90, output_file_path=output_file_path)